# Tutorial XII - Passenger Flow Control in Urban Rail

Applied Optimization with Julia

# Introduction

![](https://images.beyondsimulations.com/ao/ao_metro-metro_tutorial.svg)

Consider the depicted ‘Golden Line’ on the left with 4 different
stations A, B, C, D. For an upcoming timeframe of **10 minutes divided
into 10 periods**, the transportation demand is going to exceed the
available capacity of the network and each origin-destination pair will
be requested by at least 1 passenger. **To handle the inflow, queues
will be put in place at each metro station.**

# The Challenge

Your task is to **minimize the queues** without exceeding the available
transport capacity of **max. 100 passengers per minute of each arc**. A
safety buffer per arc is not needed. Each station layout is excellent,
with sufficient stairs and escalators. Thus, the station itself is fully
capable of handling any inflow that may result from the optimized
restricted inflow. Nonetheless, queues are still necessary as the arcs
cannot handle each input.

# 1. Flow Analysis

Suppose we are in minute 7 at the arc of (D,A). Which inflows from which
stations are going to impact the flow into the arc in this minute?

Write out the set $\mathcal{R}_{(D,A),7}$ and **shortly explain** your
answer. You can write out the set in a comment, for example like this:

``` julia
#=
{(A,B,1), (B,C,1), (C,D,1)}
=#
```

> **Tip**
>
> To solve this task, it might be helpful to work with paper and pen to
> sketch the problem.

In [1]:
#=
YOUR ANSWER BELOW
{
    (D,A,7),
    (C,A,6),
    (B,A,4),
    (D,B,7),
    (D,C,7),
    (C,B,6),
}
=#

# 2. Inflow Control

<figure>
<img
src="https://images.beyondsimulations.com/ao/ao_metro-inflow_var.png"
alt="Example: Fluctuations" />
<figcaption aria-hidden="true">Example: Fluctuations</figcaption>
</figure>

<figure>
<img
src="https://images.beyondsimulations.com/ao/ao_metro-inflow_smooth.png"
alt="Example: Smoothed" />
<figcaption aria-hidden="true">Example: Smoothed</figcaption>
</figure>

As there is a rather large fluctuation of allowed inflow between
periods, you are asked to introduce new constraints for the model. In
the first period, the inflow at each station is supposed to be
unrestricted. Thereafter, it is maximally allowed to change by 20
persons in both directions per period at each station.

Can you write out the decision variables and the additional constraints
for the model as JuMP constraints?

> **Tip**
>
> You don’t need to solve the model or define the objective function.
> You just need to constraint the fluctuations and add the appropriate
> variables.

In [3]:
using JuMP
metro_model = Model()

# YOUR CODE BELOW
set_stations = [:A, :B, :C, :D]
set_periods = 1:10
@variable(metro_model, x[i in set_stations, p in set_periods] >= 0)
@constraint(metro_model, upper_bound[i in set_stations, p in set_periods; p > 1], x[i, p] - x[i, p-1] <= 20)
@constraint(metro_model, lower_bound[i in set_stations, p in set_periods; p > 1], x[i, p] - x[i, p-1] >= -20)

# 3. Bidirectional Flow

<figure>
<img
src="https://images.beyondsimulations.com/ao/ao_metro-metro_tutorial_bidirect.svg"
alt="Bidirectional Metro Network" />
<figcaption aria-hidden="true">Bidirectional Metro Network</figcaption>
</figure>

The metro was improved and there is now the possibility to travel in
both directions. How would this change the set $\mathcal{R}_{(MD,MA),7}$
from 1.?

Write out the new set $\mathcal{R}_{(D,A),7}$ manually and shortly
explain your answer.

In [5]:
#=
# YOUR ANSWER BELOW
{
    (C, A, 6),
    (D, A, 7)
}
In case we acknowledge, that the travel time on some paths is identical, we can also include more entries such as (B,A,4).
=#

# 4. Capacity Analysis

Although the system is two-directional now, the **overall number of
trains of the metro provider has not changed**. Would the change from a
one-directional metro system to a two-directional metro system decrease
the likelihood of crowd-accidents due to insufficient arc-capacities?

Please explain your answer in a few sentences.

In [7]:
#=
# YOUR ANSWER BELOW
That does really depend on the actual demand for the different OD-pairs. Depending on the demand we can find cases where the two-directional flow is better and cases where the one-directional flow is better.

=#

# 5. Computing the Set $\mathcal{R}_{e,t}$

Can you compute the set $\mathcal{R}_{e,t}$ for the one-directional
flow? Generate a dictionary $R$ that contains $e \times t$ entries. Each
entry $r_{e,t}$ should contain a vector with all origin-destination
pairs and the corresponding time period saved as a tuple. Use the
results to check your answer from the first task.

> **Note**
>
> This task can be a bit tricky, as it is a bit of a challenge. But as
> it is the last tutorial, I figured a small challenge is fine.

In [9]:
# YOUR CODE BELOW
# Solution for one directional flow
R = Dict()

Minutes = 1:10
Stations = [:A, :B, :C, :D]
OD_pairs = [(o, d) for o in Stations, d in Stations if o != d]
Arcs = [(:A, :B), (:B, :C), (:C, :D), (:D, :A)]

dist_dict = Dict(
    (:A, :B) => 4,
    (:B, :C) => 2,
    (:C, :D) => 1,
    (:D, :A) => 1
)

# Define paths for each origin-destination pair
paths = Dict(
    (:A, :B) => [(:A, :B)],
    (:A, :C) => [(:A, :B), (:B, :C)],
    (:A, :D) => [(:A, :B), (:B, :C), (:C, :D)],
    (:B, :C) => [(:B, :C)],
    (:B, :D) => [(:B, :C), (:C, :D)],
    (:B, :A) => [(:B, :C), (:C, :D), (:D, :A)],
    (:C, :D) => [(:C, :D)],
    (:C, :A) => [(:C, :D), (:D, :A)],
    (:C, :B) => [(:C, :D), (:D, :A), (:A, :B)],
    (:D, :A) => [(:D, :A)],
    (:D, :B) => [(:D, :A), (:A, :B)],
    (:D, :C) => [(:D, :A), (:A, :B), (:B, :C)]
)

# Calculate total travel time for each OD pair
travel_times = Dict()
for (od_pair, path) in paths
    travel_times[od_pair] = sum(dist_dict[arc] for arc in path)
end
for station in Stations
    travel_times[(station, station)] = 0
end

# Display the travel times
for (od_pair, time) in travel_times
    println("Time from $(od_pair[1]) to $(od_pair[2]): $time minutes")
end

# Populate R with tuples (o, d, p) based on the given conditions
for e in Arcs
    for t in Minutes
        R[(e, t)] = [
            (o, d, tt) 
            for (o, d) in OD_pairs
            for tt in Minutes
            if e in paths[(o, d)] && (t - travel_times[(o,e[1])] == tt) && (tt >= 1) && (tt <= 10)
        ]
    end
end

# Display the computed R for verification
for e in Arcs
    for t in Minutes
        println("R[($e), $t] = ", R[(e, t)])
    end
end

Time from D to D: 0 minutes
Time from C to A: 2 minutes
Time from C to C: 0 minutes
Time from A to C: 6 minutes
Time from D to B: 5 minutes
Time from B to B: 0 minutes
Time from B to D: 3 minutes
Time from A to D: 7 minutes
Time from C to D: 1 minutes
Time from B to C: 2 minutes
Time from A to B: 4 minutes
Time from A to A: 0 minutes
Time from C to B: 6 minutes
Time from B to A: 4 minutes
Time from D to A: 1 minutes
Time from D to C: 7 minutes
R[((:A, :B)), 1] = [(:A, :B, 1), (:A, :C, 1), (:A, :D, 1)]
R[((:A, :B)), 2] = [(:A, :B, 2), (:D, :B, 1), (:A, :C, 2), (:D, :C, 1), (:A, :D, 2)]
R[((:A, :B)), 3] = [(:A, :B, 3), (:C, :B, 1), (:D, :B, 2), (:A, :C, 3), (:D, :C, 2), (:A, :D, 3)]
R[((:A, :B)), 4] = [(:A, :B, 4), (:C, :B, 2), (:D, :B, 3), (:A, :C, 4), (:D, :C, 3), (:A, :D, 4)]
R[((:A, :B)), 5] = [(:A, :B, 5), (:C, :B, 3), (:D, :B, 4), (:A, :C, 5), (:D, :C, 4), (:A, :D, 5)]
R[((:A, :B)), 6] = [(:A, :B, 6), (:C, :B, 4), (:D, :B, 5), (:A, :C, 6), (:D, :C, 5), (:A, :D, 6)]
R[((:A, :B)), 7]

In [10]:
# YOUR CODE BELOW
# Solution for bi-directional flow
R = Dict()

Minutes = 1:10
Stations = [:A, :B, :C, :D]
OD_pairs = [(o, d) for o in Stations, d in Stations if o != d]
Arcs = [
    (:A, :B), 
    (:B, :C), 
    (:C, :D), 
    (:D, :A),
    (:B, :A), 
    (:C, :B), 
    (:D, :C), 
    (:A, :D),
]

dist_dict = Dict(
    (:A, :B) => 4,
    (:B, :C) => 2,
    (:C, :D) => 1,
    (:D, :A) => 1,
    (:B, :A) => 4,
    (:C, :B) => 2,
    (:D, :C) => 1,
    (:A, :D) => 1
)

# Define paths for each shortest path origin-destination pair
paths = Dict(
    (:A, :B) => [(:A, :B)],
    (:A, :C) => [(:A, :D), (:D, :C)],
    (:A, :D) => [(:A, :D)],
    (:B, :C) => [(:B, :C)],
    (:B, :D) => [(:B, :C), (:C, :D)],
    (:B, :A) => [(:B, :A)],
    (:C, :D) => [(:C, :D)],
    (:C, :A) => [(:C, :D), (:D, :A)],
    (:C, :B) => [(:C, :B)],
    (:D, :A) => [(:D, :A)],
    (:D, :B) => [(:D, :C), (:C, :B)],
    (:D, :C) => [(:D, :C)]
)

# Calculate total travel time for each OD pair
travel_times = Dict()
for (od_pair, path) in paths
    travel_times[od_pair] = sum(dist_dict[arc] for arc in path)
end
for station in Stations
    travel_times[(station, station)] = 0
end

# Display the travel times
for (od_pair, time) in travel_times
    println("Time from $(od_pair[1]) to $(od_pair[2]): $time minutes")
end

# Populate R with tuples (o, d, p) based on the given conditions
for e in Arcs
    for t in Minutes
        R[(e, t)] = [
            (o, d, tt) 
            for (o, d) in OD_pairs
            for tt in Minutes
            if e in paths[(o, d)] && (t - travel_times[(o,e[1])] == tt) && (tt >= 1) && (tt <= 10)
        ]
    end
end

# Display the computed R for verification
for e in Arcs
    for t in Minutes
        println("R[($e), $t] = ", R[(e, t)])
    end
end

Time from D to D: 0 minutes
Time from C to A: 2 minutes
Time from C to C: 0 minutes
Time from A to C: 2 minutes
Time from D to B: 3 minutes
Time from B to B: 0 minutes
Time from B to D: 3 minutes
Time from A to D: 1 minutes
Time from C to D: 1 minutes
Time from B to C: 2 minutes
Time from A to B: 4 minutes
Time from A to A: 0 minutes
Time from C to B: 2 minutes
Time from B to A: 4 minutes
Time from D to A: 1 minutes
Time from D to C: 1 minutes
R[((:A, :B)), 1] = [(:A, :B, 1)]
R[((:A, :B)), 2] = [(:A, :B, 2)]
R[((:A, :B)), 3] = [(:A, :B, 3)]
R[((:A, :B)), 4] = [(:A, :B, 4)]
R[((:A, :B)), 5] = [(:A, :B, 5)]
R[((:A, :B)), 6] = [(:A, :B, 6)]
R[((:A, :B)), 7] = [(:A, :B, 7)]
R[((:A, :B)), 8] = [(:A, :B, 8)]
R[((:A, :B)), 9] = [(:A, :B, 9)]
R[((:A, :B)), 10] = [(:A, :B, 10)]
R[((:B, :C)), 1] = [(:B, :C, 1), (:B, :D, 1)]
R[((:B, :C)), 2] = [(:B, :C, 2), (:B, :D, 2)]
R[((:B, :C)), 3] = [(:B, :C, 3), (:B, :D, 3)]
R[((:B, :C)), 4] = [(:B, :C, 4), (:B, :D, 4)]
R[((:B, :C)), 5] = [(:B, :C, 5), (:B

------------------------------------------------------------------------

# Solutions

You will likely find solutions to most exercises online. However, I
strongly encourage you to work on these exercises independently without
searching explicitly for the exact answers to the exercises.
Understanding someone else’s solution is very different from developing
your own. Use the lecture notes and try to solve the exercises on your
own. This approach will significantly enhance your learning and
problem-solving skills.

Remember, the goal is not just to complete the exercises, but to
understand the concepts and improve your programming abilities. If you
encounter difficulties, review the lecture materials, experiment with
different approaches, and don’t hesitate to ask for clarification during
class discussions.

Later, you will find the solutions to these exercises online in the
associated GitHub repository, but we will also quickly go over them in
next week’s tutorial. To access the solutions, click on the Github
button on the lower right and search for the folder with today’s lecture
and tutorial. Alternatively, you can ask ChatGPT or Claude to explain
them to you. But please remember, the goal is not just to complete the
exercises, but to understand the concepts and improve your programming
abilities.